<span style="font-family: Arial; font-weight:bold;font-size:2em;color:#00b3e5;"> Code for scrapping data
    
<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Submitted by
    
<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Shameer.Sutar


In [18]:
import wikipedia
import csv

# Search Wikipedia for Canoo the electric vehicle company  
result = wikipedia.page("Canoo (electric vehicle manufacturer)") 

# Extract full content of the Wikipedia page
full_text = result.content

print(full_text)

# Write the scraped data to a CSV file
with open('canoo_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Overview"]) 
    writer.writerow([full_text])


Canoo Inc. is an American automotive company based in Torrance, California, that develops and manufactures electric vehicles. Canoo's research & development team is based in Michigan, in the Detroit region (Auburn Hills, Livonia), and production operations are in Justin, Texas. The company also plans to produce commercial electric vehicles such as vans for fleet, vehicle rental and ride sharing services.


== History ==
Canoo was founded in 2017 under the name Evelozcity by Stefan Krause and Ulrich Kranz. Krause worked for Deutsche Bank as its chief financial officer while Kranz worked for BMW as a senior executive. Both men met at rival EV company Faraday Future before leaving together to form their own company in 2017, due to disagreement with Faraday Future's leadership. Krause took on the role of chief executive officer at Evelozcity, and Kranz became chief technology officer. The company received its primary funding from Chinese investor Li "David" Pak-Tam/Botan and German entrepr

In [19]:
import csv
from googlesearch import search
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
from selenium.webdriver.firefox.options import Options
import random

# Function to search queries and retrieve URLs using Google search
def search_queries(query, num_results=5):
    urls = []
    try:
        counter = 0
        for url in search(query, stop=num_results):
            urls.append(url)
            counter += 1
            if counter >= num_results:
                break
    except Exception as e:
        print(f"An error occurred while fetching search results for '{query}': {e}")
    return urls

# Function to scrape data from URLs using Selenium WebDriver
def scrape_data(urls):
    scraped_data = []
    options = Options()
    options.headless = True  # Run Firefox in headless mode (without opening a browser window)
    driver = webdriver.Firefox(options=options)

    for url in urls:
        retries = 3  # Number of retries
        delay = 1  # Initial delay in seconds
        timeout = 70  # Default timeout
        if "hayekcollege.com" in url:
            timeout = 70  # Increase timeout for problematic URL
        elif "statista.com" in url:
            timeout = 70  # increase timeout for problematic URL
        driver.set_page_load_timeout(timeout)  # Set timeout dynamically based on URL

        while retries > 0:
            try:
                driver.get(url)
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Check if the soup object is None
                if soup is None:
                    print(f"Failed to parse HTML content for {url}")
                    continue

                # Extracting title from webpage
                title = soup.title.text.strip() if soup.title else "Title Not Found"

                # Extracting content
                content = None
                if soup.body:
                    content = soup.body.get_text(separator='\n').strip()

                # If no content found, log an error and continue
                if not content:
                    print(f"No content found for {url}")
                    continue

                scraped_data.append({'Title': title, 'Content': content})
                break  # Break the retry loop if successful
            except TimeoutException:
                print(f"Page load timed out while scraping {url}. Retrying...")
            except NoSuchElementException as e:
                print(f"No content found for {url}: {e}")
                break  # Break the retry loop if NoSuchElementException occurs
            except WebDriverException as e:
                print(f"WebDriver error occurred while scraping {url}: {e}")
                break  # Break the retry loop if WebDriverException occurs
            except Exception as e:
                print(f"An unexpected error occurred while scraping {url}: {e}")
                break  # Break the retry loop if unexpected error occurs
            finally:
                retries -= 1
                if retries > 0:
                    # Exponential backoff
                    delay *= 2
                    delay += random.uniform(0, 1)  # Add random jitter
                    time.sleep(delay)

    driver.quit()  # Quit the WebDriver session after scraping all URLs
    return scraped_data

# Function to save data into a text file
def save_to_text(data, filename):
    with open(filename, 'w', encoding='utf-8') as output_file:
        for item in data:
            output_file.write(f"Title: {item['Title']}\n")
            output_file.write(f"Content: {item['Content']}\n\n")

# Main function to orchestrate the process
def main():
    # Queries to search
    queries = [
        "Canoo industry overview",
        "Canoo competitors analysis",
        "Electric vehicle market trends",
        "Canoo financial performance"
    ]

    # Perform searches and scrape data
    all_data = []
    for query in queries:
        urls = search_queries(query)
        data = scrape_data(urls)
        all_data.extend(data)

    # Save scraped data into a text file
    if all_data:
        save_to_text(all_data, 'canoo_data.txt')
        print("Data successfully scraped and saved to 'canoo_data.txt'")
    else:
        print("No data scraped.")

if __name__ == "__main__":
    main()


Data successfully scraped and saved to 'canoo_data.txt'


In [20]:
import re
import csv
import requests
import PyPDF2
from googlesearch import search
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
from selenium.webdriver.firefox.options import Options
import random

# Function to search queries and retrieve URLs using Google search

def search_queries(query, num_results=5):
    urls = []
    try:
        counter = 0
        for url in search(query, stop=num_results):
            urls.append(url)
            counter += 1
            if counter >= num_results:
                break
    except Exception as e:
        print(f"An error occurred while fetching search results for '{query}': {e}")
    return urls

# Function to extract text from PDF URLs
def extract_text_from_pdf(url, filename):
    try:
        response = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(response.content)
        
        with open(filename, 'rb') as f:
            reader = PyPDF2.PdfFileReader(f)
            text = ''
            for page_num in range(reader.numPages):
                text += reader.getPage(page_num).extractText()
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return None

# Function to sanitize filenames
def sanitize_filename(title):
    return re.sub(r'[<>:"/\\|?*]', '_', title)

# Function to scrape data from URLs using Selenium WebDriver

def scrape_data(urls):
    scraped_data = []
    options = Options()
    options.headless = True  # Run Firefox in headless mode (without opening a browser window)
    driver = webdriver.Firefox(options=options)

    for url in urls:
        retries = 3  # Number of retries
        delay = 1  # Initial delay in seconds
        timeout = 70  # Default timeout
        if "pitchbook.com" in url:
            timeout = 120  # Increase timeout for problematic URL
        elif "macroaxis.com" in url:
            timeout = 120  # Increase timeout for problematic URL
        driver.set_page_load_timeout(timeout)  # Set timeout dynamically based on URL

        while retries > 0:
            try:
                driver.get(url)
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Check if the soup object is None
                if soup is None:
                    print(f"Failed to parse HTML content for {url}")
                    continue

                # Extracting title from webpage
                title = soup.title.text.strip() if soup.title else "Title Not Found"

                # Extracting content
                content = None
                if soup.body:
                    content = soup.body.get_text(separator='\n').strip()

                # If no content found, log an error and continue
                if not content:
                    print(f"No content found for {url}")
                    continue

                # Sanitize title for use as filename
                filename = f"{sanitize_filename(title)}.txt"

                # Save content to a file
                with open(filename, 'w', encoding='utf-8') as file:
                    file.write(content)

                scraped_data.append({'Title': title, 'Content': content, 'File': filename})
                break  # Break the retry loop if successful
            except TimeoutException:
                print(f"Page load timed out while scraping {url}. Retrying...")
            except NoSuchElementException as e:
                print(f"No content found for {url}: {e}")
                break  # Break the retry loop if NoSuchElementException occurs
            except WebDriverException as e:
                print(f"WebDriver error occurred while scraping {url}: {e}")
                break  # Break the retry loop if WebDriverException occurs
            except Exception as e:
                print(f"An unexpected error occurred while scraping {url}: {e}")
                break  # Break the retry loop if unexpected error occurs
            finally:
                retries -= 1
                if retries > 0:
                    # Exponential backoff
                    delay *= 2
                    delay += random.uniform(0, 1)  # Add random jitter
                    time.sleep(delay)

    driver.quit()  # Quit the WebDriver session after scraping all URLs
    return scraped_data

# Main function to orchestrate the process

def main():
    # Queries to search
    queries = [
        "Canoo industry overview",
        "Canoo competitors analysis",
        "Electric vehicle market trends",
        "Canoo financial performance"
    ]

    
    # Perform searches and scrape data
    all_data = []
    for query in queries:
        urls = search_queries(query)
        data = scrape_data(urls)
        all_data.extend(data)

    # Save scraped data into a text file
    with open('canoo_data1.txt', 'w', encoding='utf-8') as output_file:
        for item in all_data:
            output_file.write(f"Title: {item['Title']}\n")
            output_file.write(f"Content: {item['Content']}\n")
            output_file.write(f"File: {item['File']}\n\n")
    
if __name__ == "__main__":
    main()


No content found for https://www.wsj.com/market-data/quotes/GOEV/financials


**scrapping data into csv's was difficult because of fowllowing reasons hence stored in text files**

>> **Structured Data Handling:** CSV files are more suitable for structured data where each row represents a record and each column represents a field. However, scraped data from websites often come in unstructured or semi-structured formats, such as HTML or plain text. Parsing and converting such unstructured data into a structured format suitable for CSVs can be complex and error-prone.

>> **Data Integrity:** CSV files assume a tabular structure where each row has the same number of fields. In web scraping, however, the data obtained from different web pages may vary significantly in structure and content. Ensuring data integrity and consistency across different scrapes and handling variations in data format can be challenging when storing in CSV format.

>> **Special Characters and Delimiters:** CSV files use commas or other delimiters to separate fields. If the scraped data contains special characters or the same delimiter used in the content, it can lead to misinterpretation of data or parsing errors. Proper escaping or encoding of special characters is necessary to avoid such issues, adding complexity to the data storage process.

>> **Data Loss:** CSV format may not preserve all the information obtained during web scraping, especially if the data contains nested structures, rich text, or multimedia content. Storing such complex data in CSV format may result in loss of information or data truncation, leading to loss of valuable insights.

>> **Additional Processing:** After scraping data from websites, further processing such as data cleaning, normalization, or transformation may be required before storing in CSV format. This additional processing adds complexity and overhead to the data storage pipeline.

Due to these challenges, storing scraped data in text files (e.g., plain text or HTML format) is often preferred in web scraping tasks. Text files offer more flexibility in handling unstructured data, preserving the original content without loss of information, and requiring less preprocessing compared to CSV files. 


**Web scraping, or the process of extracting data from websites, can be challenging due to several reasons: and faced such problems**

>> **Website Structure Variability:** Websites are built using various technologies (HTML, CSS, JavaScript, etc.) and may have different structures and layouts. This variability makes it challenging to create a single scraping script that works effectively across all websites. Developers often need to customize scraping scripts for each website or even for different pages within the same website.

>> **Dynamic Content Loading:** Many modern websites use dynamic content loading techniques, such as AJAX or JavaScript frameworks like React or Angular. These techniques asynchronously fetch data from the server and update the webpage without reloading the entire page. Traditional web scraping tools may struggle to interact with and extract data from dynamically loaded content, requiring more advanced techniques such as headless browsers or browser automation.

>> **Anti-Scraping Measures:** To prevent automated access and scraping, websites may implement various anti-scraping measures such as CAPTCHA challenges, rate limiting, IP blocking, or user-agent detection. Dealing with these measures requires sophisticated scraping techniques and may involve rotating IP addresses, using proxy servers, or employing CAPTCHA-solving services.

>> **Session Management and Authentication:** Some websites require users to log in or maintain session cookies to access certain pages or data. Scraping authenticated content or handling session management adds complexity to the scraping process, as it involves handling cookies, managing user sessions, and dealing with login forms.

>> **Legal and Ethical Concerns:** Web scraping may raise legal and ethical concerns, as scraping copyrighted or sensitive data without permission may violate website terms of service or copyright laws. Scraping large amounts of data from a website may also put strain on the website's servers and impact its performance, leading to potential legal issues or ethical considerations.

>> **Data Quality and Consistency:** The quality and consistency of scraped data can vary widely depending on factors such as website design, data formatting, and changes in website layout or content. Scraped data may contain errors, missing information, or inconsistencies, requiring data cleaning and preprocessing before analysis or use.

>> **Maintenance Overhead:** Websites are frequently updated, redesigned, or restructured, which can break existing scraping scripts. Maintaining and updating scraping scripts to adapt to changes in website structure or content requires ongoing effort and monitoring.

Despite these challenges, web scraping remains a powerful tool for extracting valuable data from the web for various purposes such as market research, competitive analysis, and data-driven decision-making. Overcoming these challenges often involves a combination of technical expertise, adaptability, and adherence to legal and ethical standards.


### Apart from these chanlleges i was able to scrape some amount of data using above coding scripts about canno inc company and made a report  based on the scrapped data and completed the task as yu requested sir.